# Inheritance

In [1]:
class A:
    password = 'Joker' #class attribute
    def __init__(self, name):
        self.name = name #instance attribute
    def greet(self):
        print(f'Hello {self.name}')

In [2]:
class B(A):
    pass

In [3]:
class_b = B('VN Pikachu')
class_b.greet()

Hello VN Pikachu


# Overriding

In [4]:
class C(A):
    def greet(self):
        print(f'Mother fucker you, {self.name}')

In [5]:
class_c = C('Saklo Patel')
class_c.greet()

Mother fucker you, Saklo Patel


when overriding, the original method can still be accessed, but we have to do it by calling the method directly with class name:

In [6]:
A.greet(class_c)

Hello Saklo Patel


let's reuse, then override the greet method:

In [7]:
class D(A):
    def greet(self):
        A.greet(self)
        print('overriding by reusing the original method')

In [8]:
class_d = D('Tank Cao')
class_d.greet()

Hello Tank Cao
overriding by reusing the original method


# Example

In [9]:
import random

In [10]:
class Robot:
    health_level = 0.6
    forbidden_names = ['VN Pikachu', 'Tank Cao']
    def __init__(self, name):
        self.name = name
        self.health = random.random()
    @property
    def name(self):
        return self.__name
    @name.setter
    def name(self, name):
        if name in Robot.forbidden_names:
            self.__name = 'Guest'
        else:
            self.__name = name
    def __add__(self, other):
        return type(self)(self.name + '-' + other.name)
    def __str__(self):
        return f'Robot: {self.name}'
    def needs_a_nurse(self):
        return self.health <= Robot.health_level
    def say_hi(self):
        print(f'Hi, I am {self.name} with health of {self.health}')

In [11]:
first_generation = (
Robot('VN Pikachu'), Robot('Monkey'), Robot('Zabrah'))

In [12]:
first_generation[0].name

'Guest'

In [13]:
first_generation[1].name

'Monkey'

In [14]:
first_generation[0].say_hi()

Hi, I am Guest with health of 0.5587474014384481


In [15]:
first_generation[1].say_hi()

Hi, I am Monkey with health of 0.6435086933430876


In [16]:
first_generation[0].needs_a_nurse()

True

In [17]:
first_generation[1].needs_a_nurse()

False

In [18]:
gen1 = first_generation
babies = [gen1[0] + gen1[1], gen1[1] + gen1[2]]
babies.append(babies[0] + babies[1])

for baby in babies:
    print(baby.name)
    baby.say_hi()

Guest-Monkey
Hi, I am Guest-Monkey with health of 0.7144289970721349
Monkey-Zabrah
Hi, I am Monkey-Zabrah with health of 0.9828428313429468
Guest-Monkey-Monkey-Zabrah
Hi, I am Guest-Monkey-Monkey-Zabrah with health of 0.6926467497069718


In [19]:
class NursingRobot(Robot):
    def __init__(self, name, healing_power = 0.8):
        super().__init__(name)
    def say_hi(self):
        print(f'I am {self.name}, everything will be ok!')
    def say_hi_to_doc(self):
        Robot.say_hi(self)
    def heal(self, target):
        target.health = max(target.health, self.healing_power)
        print('Action completed!')

In [20]:
nurse1 = NursingRobot('Tank Cao')
nurse2 = NursingRobot('Meomeo888')

In [21]:
nurse1.name

'Guest'

In [22]:
nurse2.name

'Meomeo888'

In [23]:
nurse1.say_hi()

I am Guest, everything will be ok!


In [24]:
nurse2.say_hi()

I am Meomeo888, everything will be ok!


In [25]:
nurse1.say_hi_to_doc()

Hi, I am Guest with health of 0.35822439246741056


In [26]:
nurse2.say_hi_to_doc()

Hi, I am Meomeo888 with health of 0.13655641886231107


In [27]:
nurse3 = nurse1 + nurse2
nurse3.name

'Guest-Meomeo888'

In [28]:
nurse3.say_hi()

I am Guest-Meomeo888, everything will be ok!


In [29]:
nurse3.say_hi_to_doc()

Hi, I am Guest-Meomeo888 with health of 0.05015317358424454


# Multiple Inheritance

```python
class DerivedClassName(Base1, Base2, Base3):
    <statement-1>
    .
    .
    .
    <statement-N>
```

For most purposes, in the simplest cases, you can think of the search for attributes inherited from a parent class as depth-first, left-to-right, not searching twice in the same class where there is an overlap in the hierarchy. Thus, if an attribute is not found in DerivedClassName, it is searched for in Base1, then (recursively) in the base classes of Base1, and if it was not found there, it was searched for in Base2, and so on.

# The diamond problem


The Diamond Problem or the ,,deadly diamond of death''
Diamond Problem

The "diamond problem" (sometimes referred as the "deadly diamond of death") is the generally used term for an ambiguity that arises when two classes B and C inherit from a superclass A, and another class D inherits from both B and C. If there is a method "m" in A that B or C (or even both of them) has overridden, and furthermore, if it does not override this method, then the question is which version of the method does D inherit? It could be the one from A, B or C.

In [1]:
class A:
    def m(self):
        print('A')

class B(A):
    def m(self):
        print('B')
class C(A):
    def m(self):
        print('C')
class D(B, C):
    def m(self):
        print('D')


Now let's assume that the method m of D should execute the code of m of B, C and A as well, when it is called. We could implement it like this:

In [2]:
class D(B, C):
    def m(self):
        print('D')
        C.m(self)
        B.m(self)
        A.m(self)
        
d = D()
d.m()

D
C
B
A


But it turns out once more that things are more complicated than they seem. How can we cope with the situation, if both m of B and m of C will have to call m of A as well. In this case, we have to take away the call A.m(self) from m in D. The code might look like this, but there is still a bug lurking in it:

In [3]:
class A:
    def m(self):
        print('A')
        
class B(A):
    def m(self):
        print('B')
        A.m(self)
class C(A):
    def m(self):
        print('C')
        A.m(self)
        
class D(B,C):
    def m(self):
        print('D')
        B.m(self)
        C.m(self)

The bug is that the method m of A will be called twice:

In [4]:
d = D()
d.m()

D
B
A
C
A


A non-pythonic way

In [5]:
class A:
    def m(self):
        print("m of A called")

class B(A):
    def _m(self):
        print("m of B called")
    def m(self):
        self._m()
        A.m(self)
    
class C(A):
    def _m(self):
        print("m of C called")
    def m(self):
        self._m()
        A.m(self)

class D(B,C):
    def m(self):
        print("m of D called")
        B._m(self)
        C._m(self)
        A.m(self)
        
d = D()
d.m()

m of D called
m of B called
m of C called
m of A called


The pythonic way:

In [6]:
class A:
    def m(self):
        print('A')
class B(A):
    def m(self):
        print('B')
        super().m()
class C(A):
    def m(self):
        print('C')
        super().m()
class D(B, C):
    def m(self):
        print('D')
        super().m()
        
d = D()
d.m()

D
B
C
A


he super function is often used when instances are initialized with the `__init__` method:

In [9]:
class A:
    def __init__(self):
        print('Init A')
        pass
class B(A):
    def __init__(self):
        print('Init B')
        super().__init__()
class C(A):
    def __init__(self):
        print('Init C')
        super().__init__()
class D(B,C):
    def __init__(self):
        print('Init D')
        super().__init__()

d = D()

Init D
Init B
Init C
Init A
